In [ ]:
import numpy as np
import pandas as pd
import datetime
import scipy.stats as stats
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr

# European Call Options valuation

The risk neutral prcing methodology tells us that:

*   Value of an option = risk neutral expectation of its discounted payoff

We can estimate this expectation by computing the average of a large nuber off discounted payoffs. For a particular simulation i:

$$ C_{0,i} = exp( - \int_{0}^{T} r_s ds) C_{T,i} = exp( - rT) C_{T,i} $$

If we repeat the simulation M times, we can average the outcomes:
$$ \hat{C_{0}} = \frac{1}{M} \sum_{i=1}^M  C_{0,i}$$

## Standard Error SE($\hat{C_{0}}$)
$\hat{C_{0}}$ is an estimate of the true value of the option $C_{0}$ with error due to the fact that we are taking an average of randomly generated samples, and therefore the calculation is itself random. A of this error is the standard deviation of $\hat{C_{0}}$ called the standard errorr. This can be estimated as the standard deviation of $C_{0}$, divided by the number of samples $M$.

$$\hat{C_{0}} = \frac{\sigma(C_{0,i})}{\sqrt{M}}$$

$$\sigma(C_{0,i} = \sqrt{\frac{1}{M-1} \sum_{i=1}^M(C_{0,i} - \hat{C_{0}})^2}$$


# European calls in the BS world

If we consider $X : = log S$ then the asset price process verifies this SDE:

$$dx_t = \nu dt + \sigma  dZ_t  , \nu =r - \frac{1}{2}\sigma ^ 2$$

Discretisizing : $ \Delta x = \nu \Delta t + \sigma \Delta Z$

$$x_{t+\Delta t} = x_t + \nu \Delta t + \sigma(Z_{t+\Delta t} - Z_t )$$

in terms f the stock price $S_t$:
 $$S_{t+\Delta t} = S_t exp(\nu \Delta t + \sigma(Z_{t+\Delta t} - Z_t ))$$

Where $(z_{t+\Delta t}- z_t) \sim N(0,\Delta t) \sim \sqrt{\Delta t} N(0,1) \sim \sqrt{\Delta t} \epsilon_i$





In [ ]:
S = 101.15
K = 98.01
vol = 0.0991
r = 0.01
N = 10
M = 1000

market_value = 3.86
T = ((datetime.date(2024,5,17)-datetime.date.today()).days+1)/365 # Time in years
print(T)

0.07945205479452055


In [ ]:
# Precompute constants
dt = T/N
nudt = (r - 0.5*vol**2)*dt
volsdt = vol*np.sqrt(dt)
lnS = np.log(S)

# Standard Error Placeholders
sum_CT = 0
sum_CT2 = 0

# Monte Carlo Method
for i in range(M):
    lnSt = lnS
    for j in range(N):
        epsilon = np.random.normal()
        lnSt = lnSt + nudt + volsdt*epsilon

    ST = np.exp(lnSt)
    CT = max(0, ST - K)
    sum_CT = sum_CT + CT
    sum_CT2 = sum_CT2 + CT*CT

# Compute Expectation and SE
C0 = np.exp(-r*T)*sum_CT/M
sigma = np.sqrt( (sum_CT2 - sum_CT*sum_CT/M)*np.exp(-2*r*T) / (M-1) )
SE = sigma/np.sqrt(M)

print("Call value is ${0} with SE +/- {1}".format(np.round(C0,2),np.round(SE,2)))

Call value is $3.28 with SE +/- 0.08


# Monte Carlo Variance Reduction Methods - Antithetic

Unfortunately, although a great method for approximating option values with complex payoffs or high dimensionality, in order to get an acceptably accurate estimate we must perform a large number of simulations M. Instead we can lean on Variance Reduction methods which work on the same principles as that of hedging an option position. i.e. the variability of a hedged option portfolio will have a smaller variance than that of it's unhedged counterpart.


## Antithetic Variates

Let's write an option on asset $S_1$ and another option on asset $S_2$ that is perfectly negatively correlated with $S_1$ and which currently has the same price. $S_1$ and $S_2$ satisfy the following Stochastic Differential Equations:

$\large dS_{1,t} = rdS_{1,t}dt+\sigma dS_{1,t}dz_t$

$\large dS_{2,t} = rdS_{2,t}dt-\sigma dS_{2,t}dz_t$

Since the price and volatility of the two assets are identical, so is the value of these two options. However, the variance of a portfolio pay-off containing both of these contracts is much less than the variance of the pay-off of each individual contract. In essence we are removing the large spike in probability distribution of a single contract pay-off. i.e. Basic Intuition: when one option pays out, the other does not.


## Implementation of Antithetic Variate

To implement an antithetic variate we create a hypothetical asset which is perfectly negatively correlated with the original asset. Implementation is very simple, and if we consider the example of the European Call Option (as in last weeks video). Our simulated pay-offs are under the following $S_t$ dynamics:

$\large S_{t+\Delta t} = S_{t} \exp( \nu \Delta t + \sigma (z_{t+\Delta t}- z_t) )$

Where $(z_{t+\Delta t}- z_t) \sim N(0,\Delta t) \sim \sqrt{\Delta t} N(0,1) \sim \sqrt{\Delta t} \epsilon_i$

### Contract Simulation

- $\large C_{T,i} = max(0, S \exp( \nu \Delta T + \sigma \sqrt{T} (\epsilon_i) ) - K)$

- $\large \bar{C}_{T,i} = max(0, S \exp( \nu \Delta T + \sigma \sqrt{T} (-\epsilon_i) ) - K)$


In [ ]:
# Standard Error Placeholders
sum_CT = 0
sum_CT2 = 0

# Monte Carlo Method
for i in range(M):
    lnSt1 = lnS
    lnSt2 = lnS
    for j in range(N):
        # Perfectly Negatively Correlated Assets
        epsilon = np.random.normal()
        lnSt1 = lnSt1 + nudt + volsdt*epsilon
        lnSt2 = lnSt2 + nudt - volsdt*epsilon

    ST1 = np.exp(lnSt1)
    ST2 = np.exp(lnSt2)
    CT = 0.5 * ( max(0, ST1 - K) + max(0, ST2 - K) )
    sum_CT = sum_CT + CT
    sum_CT2 = sum_CT2 + CT*CT

# Compute Expectation and SE
C0 = np.exp(-r*T)*sum_CT/M
sigma = np.sqrt( (sum_CT2 - sum_CT*sum_CT/M)*np.exp(-2*r*T) / (M-1) )
SE = sigma/np.sqrt(M)

print("Call value is ${0} with SE +/- {1}".format(np.round(C0,2),np.round(SE,2)))


Call value is $3.38 with SE +/- 0.01


# Monte Carlo Variance Reduction Methods - Control Variates


In [ ]:
!pip install py_vollib

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 5.8 MB/s eta 0:00:00
  Created wheel for py_vollib: filename=py_vollib-1.0.1-py3-none-any.whl size=62829 sha256=080d80eebfc9543b9d5446a66aeada5d3bf7744504b051e978674bf1e346eb99
  Stored in directory: /root/.cache/pip/wheels/84/0c/fc/b68506eda40cccaeb0013be550ba904d253ec90eae2b156052
  Created wheel for py_lets_be_rational: filename=py_lets_be_rational-1.0.1-py3-none-any.whl size=24450 sha256=f22e30574d6e12a96f5f012125aaadff3235d84c5f519002db93de1b701495df
  Stored in directory: /root/.cache/pip/wheels/32/d9/4f/360d4b0b9ec7c65583ec7c8e72fd2158c97ad04fbcb932891c
Successfully built py_vollib py_lets_be_rational


In [ ]:
from py_vollib.black_scholes.implied_volatility import implied_volatility as iv
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import vega, delta

/usr/local/lib/python3.10/dist-packages/py_lets_be_rational/numba_helper.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  return jit(*jit_args, **jit_kwargs)(fun)


## Control Variates & Hedging

The probability distribution of an option pay-off after delta hedging has a smaller standard deviation compared to its portfolio's unhedged counterpart. Let's consider the dynamics of a discretely rebalanced delta hedge:

$\large C_{t_0}\exp{rT} - [\sum^{N}_{i=0} ( \frac{\delta C_{t_i}}{\delta S} - \frac{\delta C_{t_{i-1}}}{\delta S} ) S_t\exp{r(T-t_i)}] = C_T + \eta$

The first term is the forward value of the premium recieved for writing the option, the second term represents t
he cash flows from rebalancing the headge at each time $t$, and the third term is the pay-off of the option and the hedging error $\eta$.

Now if we expand the terms in the square brackets:

$\frac{\delta C_{t_0}}{\delta S}S_{t_0}\exp{r(T-t_0)} + \frac{\delta C_{t_1}}{\delta S}S_{t_0}\exp{r(T-t_1)} + ... + \frac{\delta C_{t_{N-1}}}{\delta S}S_{t_{N-1}}\exp{r(T-t_{N-1})} + \frac{\delta C_N}{\delta S}S_{t_N}$

$-\frac{\delta C_{t_0}}{\delta S}S_{t_1}\exp{r(T-t_1)} - \frac{\delta C_{t_1}}{\delta S}S_{t_2}\exp{r(T-t_2)} - ... - \frac{\delta C_{t_{N-1}}}{\delta S}S_N$

Group terms with $\frac{\delta C_{t_i}}{\delta S}$ at the same time steps.

$-\frac{\delta C_{t_0}}{\delta S}(S_{t_1}-S_{t_0}\exp{r\Delta t})\exp{r(T-t_1)} - \frac{\delta C_{t_1}}{\delta S}(S_{t_2}-S_{t_1}\exp{r\Delta t})\exp{r(T-t_2)} - ... - \frac{\delta C_{t_{N-1}}}{\delta S}(S_{t_N}-S_{t_{N-1}}\exp{r\Delta t}) + \frac{\delta C_N}{\delta S}S_{t_N}$

If we assume the last term equals 0, which is equivalent to not buying the final delta amount of the asset, but simpluing liquidating the underlying holdings from the previous rebalancing date into cash, then the portfolio becomes:

$\large C_{t_0}\exp{rT} = C_T - [\sum^{N-1}_{i=0} \frac{\delta C_{t_i}}{\Delta S}(S_{t_{i+1}} - S_{t_i}\exp{r\delta t})\exp{r(T-t_{i+1})} ] + \eta$

The term in the square brackets (the delta hedge) is a delta-based martingale control variate ($cv_1$). This can be written as:

$\large cv_1 = \sum^{N-1}_{i=0} \frac{\delta C_{t_i}}{\delta S}(S_{t_{i+1}} - {\mathbb E}[S_{t_i}])\exp{r(T-t_{i+1})}$

$\large C_{t_0}\exp{rT} = C_T - cv_1 + \eta$

## Gamma Based Control Variate

The control variate is a random variable whose expected value we know, which is correlated with the varaible we are trying to estimate.

In the same way as for $cv_1$ we can create other control variates, which are equivalent to other hedges.

For example a gamma-based control variate ($cv_2$):

$\large cv_2 = \sum^{N-1}_{i=0} \frac{\delta^2 C_{t_i}}{\delta S^2}((\Delta S_{t_{i+1}})^2 - {\mathbb E}[(\Delta S_{t_i})^2])\exp{r(T-t_{i+1})}$

Where ${\mathbb E}[(\Delta S_{t_i})^2] = S_{t_i}^2 (\exp([2r+\sigma^2]\Delta t_i)-2\exp(r\Delta t_i)+1)$

## General Control Variate Equation

For J control variates we have:

$ \Large C_0\exp(rT) = C_T - \sum^J_{i=j}\beta_j cv_j + \eta$

where
- $\beta_j$ are factors to account for the "true" linear relationship between the option pay-off and the control variate $cv_j$
- $\eta$ accounts for errors:
    - discrete rebalancing
    - approximations in hedge sensitivities (calc. delta / gamma)
    
    
Option price as the sum of the linear relationships with J control variates
    
$ \large C_T =\beta_0 + \sum^J_{i=j}\beta_j cv_j + \eta$

where $\beta_0 = C_0\exp(rT)$ is the forward price of the option

If we perform M simulations at discrete time intervals N we can regard the pay-offs and control variates as samples of the linear relationship with some noise. We can estimate the true relationship between control variates and option pay-offs with least-squares regression:

$\beta = (X'X)^{-1}X'Y$

We don't want biased estimates of $\beta_j$ so these should be precomputed by least-squares regression to establish the relationship between types of control variates and options first. These estaimates of $\beta_j$ values can then be used for $cv_j$ for pricing any option.  

## Implementation of Delta-based Control Variates

$\large cv_1 = \sum^{N-1}_{i=0} \frac{\delta C_{t_i}}{\delta S}(S_{t_{i+1}} - {\mathbb E}[S_{t_i}])\exp{r(T-t_{i+1})}$

$\large C_{t_0}\exp{rT} = C_T + \beta_1 cv_1 + \eta$


where with GBM dynamics:
- ${\mathbb E}[S_{t_i}] = S_{t_{i-1}} \exp (r \Delta t_i)$
- $\beta_1 = -1$ which is the appropriate value where we have exact delta for European Option

In [ ]:
# Precompute constants
N = 10
dt = T/N
nudt = (r - 0.5*vol**2)*dt
volsdt = vol*np.sqrt(dt)
erdt = np.exp(r*dt)

beta1 = -1

# Standard Error Placeholders
sum_CT = 0
sum_CT2 = 0

# Monte Carlo Method
for i in range(M):
    St = S
    cv = 0
    for j in range(N):
        epsilon = np.random.normal()
        deltaSt = delta('c', St, K, T-j*dt, r, vol)
        Stn = St*np.exp( nudt + volsdt*epsilon )
        cv = cv + deltaSt*(Stn - St*erdt)
        St = Stn

    CT = max(0, St - K) + beta1*cv
    sum_CT = sum_CT + CT
    sum_CT2 = sum_CT2 + CT*CT

# Compute Expectation and SE
C0 = np.exp(-r*T)*sum_CT/M
sigma = np.sqrt( (sum_CT2 - sum_CT*sum_CT/M)*np.exp(-2*r*T) / (M-1) )
SE = sigma/np.sqrt(M)

print("Call value is ${0} with SE +/- {1}".format(np.round(C0,2),np.round(SE,3)))

Call value is $3.38 with SE +/- 0.006
